In [3]:
import numpy as np;
import cv2;
import matplotlib.pyplot as plt;
import os;
import glob;
from tqdm import tqdm;
import open3d as o3d;

all_bmp_files = glob.glob("./calib_2/*.bmp")
file_list = sorted(all_bmp_files, key=lambda p: int(os.path.splitext(os.path.basename(p))[0]))
deg = -3.05
raw = cv2.imread(file_list[0], cv2.IMREAD_GRAYSCALE)
raw = cv2.copyMakeBorder(raw,1000,1000,1000,1000,cv2.BORDER_CONSTANT)
center = (0,0)
rotate_matrix = cv2.getRotationMatrix2D(center, deg, 1)
raw = cv2.warpAffine(raw, rotate_matrix, (-1, -1))
raw = cv2.cvtColor(raw, cv2.COLOR_GRAY2BGR)

points_channel = []
for img_file in file_list:
    points_this_img = []
    ok_blob_num = 0
    src = cv2.imread(img_file, cv2.IMREAD_GRAYSCALE)
    src = cv2.copyMakeBorder(src,1000,1000,1000,1000,cv2.BORDER_CONSTANT)
    center = (0,0)
    rotate_matrix = cv2.getRotationMatrix2D(center, deg, 1)
    src = cv2.warpAffine(src, rotate_matrix, (-1, -1))
    debug_img = cv2.cvtColor(src, cv2.COLOR_GRAY2BGR)
    _, src = cv2.threshold(src, 30, 255, cv2.THRESH_BINARY)
    num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(src, connectivity=8)
    for i in range(1, num_labels):
        area = stats[i, cv2.CC_STAT_AREA]
        if stats[i, cv2.CC_STAT_AREA] < 20 or stats[i, cv2.CC_STAT_AREA] > 200:
            continue
        x, y, w, h, area = stats[i]
        cx, cy = centroids[i]
        cv2.circle(debug_img, (int(cx), int(cy)), 1, (0, 255, 0), -1)
        cv2.circle(raw, (int(cx), int(cy)), 1, (0, 255, 0), -1)
        points_this_img.append(np.array([cx, cy]))
        ok_blob_num += 1
    cv2.imwrite("./blob_pos/" + os.path.basename(img_file), debug_img);
    points_channel.append(np.array(points_this_img))
points_channel = np.array(points_channel)
cv2.imwrite("./blob_pos/raw.bmp", raw);

for channel in points_channel:
    channel -= np.array([0,15])

In [9]:
import numpy as np;
import cv2;
import matplotlib.pyplot as plt
import os;
import glob;
from tqdm import tqdm;
import open3d as o3d;

points_re = []
file = "./recon/11.bmp"
# file = "./imgs3/10.bmp"
img_re = cv2.imread(file, cv2.IMREAD_GRAYSCALE)
img_re = cv2.copyMakeBorder(img_re,1000,1000,1000,1000,cv2.BORDER_CONSTANT)
center = (0,0)
rotate_matrix = cv2.getRotationMatrix2D(center, deg, 1)
img_re = cv2.warpAffine(img_re, rotate_matrix, (-1, -1))
thres = 35
_, bin_img = cv2.threshold(img_re, thres, 255, cv2.THRESH_BINARY)
num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(bin_img, connectivity=8)
img_re_color = cv2.cvtColor(img_re, cv2.COLOR_GRAY2BGR)
ok_blob_num = 0
for i in range(1, num_labels):
    area = stats[i, cv2.CC_STAT_AREA]
    if area < 20 or area > 200:
        continue
    ok_blob_num += 1
    x, y, w, h, area = stats[i]
    cx, cy = centroids[i]
    cv2.circle(img_re_color, (int(cx), int(cy)), 1, (0, 0, 255), -1)
    points_re.append(np.array([cx, cy]))
print("ok_blob_num: ",ok_blob_num)
points_re = np.array(points_re)

## corresponding map
ref_points = points_channel[0]
from scipy.spatial import cKDTree
pc = []

for p in points_re:
    dx = p[0] - ref_points[:,0]
    dy = p[1] - ref_points[:,1]
    
    idx = np.where((np.abs(dx) < 3)  & (dy >= 0) & (dy <= 100))
    if idx[0].size == 0:
        print("no corresponding point")
        continue
    
    closest_point = ref_points[idx[0]][0]
    x = closest_point[0] * 3.45 / 1000
    y = closest_point[1] * 3.45 / 1000
    z = (p[1] - closest_point[1]) / 75 * (-2.4)
    # print("(", p[0] - closest_point[0], p[1] - closest_point[1], ") depth >> ", z)
    pc.append([x, y, z])

pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(pc)
o3d.visualization.draw_geometries([pcd], window_name="pc_window")
# o3d.io.write_point_cloud(os.path.basename(file).split(".")[0] + ".pcd", pcd)

ok_blob_num:  13055


In [ ]:
file_list = sorted(all_bmp_files, key=lambda p: int(os.path.splitext(os.path.basename(p))[0]))
deg = -3.05
raw = cv2.imread(file_list[0], cv2.IMREAD_GRAYSCALE)
raw = cv2.copyMakeBorder(raw,1000,1000,1000,1000,cv2.BORDER_CONSTANT)
center = (0,0)
rotate_matrix = cv2.getRotationMatrix2D(center, deg, 1)
raw = cv2.warpAffine(raw, rotate_matrix, (-1, -1))
raw = cv2.cvtColor(raw, cv2.COLOR_GRAY2BGR)
for cx,cy in np.vstack(points_channel):
    # cv2.circle(debug_img, (int(cx), int(cy)), 1, (0, 255, 0), -1)
    raw[int(cy), int(cx)] = (0,255,0)
for cx,cy in points_re:
    # cv2.circle(raw, (int(cx), int(cy)), 2, (0, 0, 255), -1)
    raw[int(cy), int(cx)] = (0,0,255)
cv2.imwrite("./raw.bmp",raw)